In [11]:
# import modules
import os
import re

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
import pandas as pd

In [12]:
saved_file = "/home/mingi/data/chartgemma/one_by_one_results/horizontal.jsonl"

In [13]:
files = [json.loads(line) for line in open(saved_file, "r")]

In [14]:
files[150]["text"]
# files[150]["value_list"]
files[140]

{'image_file': '5670.png',
 'text': ['The value of 2019 is 1,800 GBP per metric ton.',
  'The value of 2018 is 1450.',
  'The value of 2017 is 1,500.',
  'The price in 2016 is approximately 1,300 GBP per metric ton.',
  'The value of 2015 is approximately 1,300 GBP per metric ton.',
  'The value of 2014** is 1,600 GBP per metric ton.',
  'The value of 2013 is 1,600 GBP per metric ton.',
  'The value of 2012 is 1,400 GBP per metric ton.',
  '2011 = 350',
  'The value of 2010 is approximately 1,400 GBP per metric ton.',
  'The price in 2009 is approximately 1,400 GBP per metric ton.'],
 'value_list': [1518.2,
  1500.7,
  1600.8,
  1316.0,
  1357.0,
  1595.1,
  1651.5,
  1501.9,
  1447.4,
  1418.0,
  1457.9]}

In [15]:
# from pprint import pprint

# i = 105
# DIRECTION = "vertical"
# saved_file = f"/home/mingi/data/chartgemma/one_by_one_results/{DIRECTION}.jsonl"
# files = [json.loads(line) for line in open(saved_file, "r")]
# image = Image.open(
#     os.path.join(
#         f"/home/mingi/data/vistext/data/test_images/{DIRECTION}", files[i]["image_file"]
#     )
# ).convert("RGB")

# plt.imshow(image)
# plt.plot()
# pprint(files[i]["image_file"])
# pprint(files[i]["text"])
# pprint(files[i]["value_list"])

In [16]:
def extract_last_number(strings):
    result = []
    for string in strings:
        if (
            "not" in string.lower()
            or "no" in string.lower()
            or "Please" in string.lower()
            or "What" in string.lower()
        ):
            result.append(0)
        else:
            # Find all numbers (including negative numbers) in the string
            # numbers = re.findall(r"-?\d{1,3}(?:,\d{3})*(?:\.\d+)?", string)
            numbers = re.findall(r"-?\d+(?:,\d{3})*(?:\.\d+)?", string)
            numbers = [
                num for num in numbers if len(num.replace(",", "").split(".")[0]) <= 10
            ]
            if numbers:
                # Convert the last found number to an appropriate type (int or float) and add to the result
                last_number_str = numbers[-1].replace(",", "")
                last_number = (
                    float(last_number_str)
                    if "." in last_number_str
                    else int(last_number_str)
                )
                result.append(last_number)
            else:
                result.append(0)  # If no numbers found, return 0
    return result

In [17]:
extract_last_number(files[140]["text"])

[1800, 1450, 1500, 1300, 1300, 1600, 1600, 1400, 350, 1400, 1400]

In [18]:
def eval_metric(gt_values, values):

    l = len(gt_values)
    sum_error_rates = 0
    for gt_value, value in zip(gt_values, values):
        if isinstance(gt_value, str):
            try:
                gt_value = float(gt_value)
            except ValueError:
                gt_value = 0

        error = abs(gt_value - value)

        if gt_value == 0:
            l = l - 1
            continue

        error_rate = error / (abs(gt_value))
        error_rate = min(error_rate, 1)
        sum_error_rates = sum_error_rates + error_rate

    avg_error_rates = sum_error_rates / l

    return avg_error_rates

In [19]:
# Evaluation
sum = 0
for file in files:
    texts = file["text"]

    gt_values = file["value_list"]
    values = extract_last_number(texts)

    metric = eval_metric(gt_values, values)

    sum = sum + metric

avg = sum / len(files)

In [20]:
avg

0.41912798045963867